Group 18 Project Proposal
Title: Vancouver Foods

In [8]:
library(rvest)
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()         masks stats::filter()
✖ readr::guess_encoding() masks rvest::guess_encoding()
✖ dplyr::lag()            masks stats::lag()


In [11]:
#reading data
URL <- 'https://raw.githubusercontent.com/wmma2/group_18_project/main/data.csv'

van_url <- read_csv(URL)
data <- html_text(van_url)

data

New names:
• `Rating` -> `Rating...3`
• `Rating` -> `Rating...15`
Rows: 384 Columns: 17
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (14): Name of Restaurent, Type of Cusine, Cost, Landmark, Opening time, ...
dbl  (2): Rating...3, Rating...15
num  (1): Total No of Ratings

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ERROR: Error in UseMethod("xml_text"): no applicable method for 'xml_text' applied to an object of class "c('spec_tbl_df', 'tbl_df', 'tbl', 'data.frame')"


In [ ]:
Download dataset
Upload to git
If URL starts with 
Click RAW - gives you CSV file as a text file 
Make it so it starts with 'raw'